In [1]:
import numpy as np
import pandas as pd
import Nio
import datetime
import os
import math

# Dynamical Data Processing

This notebook calculates the target predictor variables from the downloaded subset of the NCEP FNL Reanalysis (1999-) dataset. The subset contains, from 1999-08-01 00:00 to 2019-12-31 23:00 and covering longitudes 53E to 164W, latitudes 8S to 55N, the following variables:

- geopotential height, 500 mb

- u- and v-components of wind, at 1000mb, 850mb, 500mb and 200mb levels

- relative humidity, at 300-500 and 750-800mb

- temperature, at surface and 200 mb

- potential temperature, at sigma level 0.995

- absolute vorticity, at 850mb

This notebook takes, most notably, PyNIO as a prerequisite, and works only with a Python 2 kernel.

In [2]:
import pickle
import csv

def save_dict_to_pickle(data, filename):
    '''
    Takes in a dictionary and a full filename (with extensions) and writes the dictionary to the specified file as pickles.
    '''
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
        
def save_dict_to_csv(data, filename):
    '''
    Takes in dictionary and a full filename (with extensions) and writes the dictionary to the specified file as a CSV (no header).
    '''
    with open(filename, 'wb') as f:
        w = csv.writer(f)
        w.writerows(data.items())

### Geopotential height

Preprocessing targets:
- WNPSH intensity index: 10°–60°N, 100°E–180°, >5870gpm, average value

- WNPSH area index: 10°–60°N, 100°E–180°, >5870gpm, grids count

- WNPSH westward extension index: 5870gpm contour westmost longitude

- westerly index: H35 - H55 from 100E to 180E

all calculated for each available time step.

Missing value considerations:
- if area index == 0,
    then no intensity/westward extension indices
    
- there are disgustingly many missing *files*.
  - 534 GRIB1 files and 17634 GRIB2 files are available, adding up to a total of 18168 files (NCEP FNL subset API claims 11542 only).
  
  - This will be the upper limit of the final dataset size.
  
Because of the missing files issue, alternative data sources may be needed.

In [89]:
# bookkeeping
wnpsh_area_indices = dict()
wnpsh_intensity_indices = dict()
wnpsh_extension_indices = dict()
westerly_indices = dict()

In [91]:
# start from this date
data_time = datetime.datetime(1999, 8, 1, 0, 0, 0)
# advance at 6 hour time steps
delta_fixee = datetime.timedelta(hours=6)

num_ok = 0
num_expected = 0
num_ng = 0

# main loop
# repeat until all data are exhausted (time limit reached)
while data_time < datetime.datetime(2019, 12, 31, 23, 0, 0):
    
    num_expected += 1

    # GRIB1 files are available until 6 Dec 2007 0600 (UTC), 
    # the variable we need in that case is HGT_3_ISBL
    # modify accordingly for GRIB2
    extension = "grib1"
    var_name = "HGT_3_ISBL"
    latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
    if data_time > datetime.datetime(2007, 12, 6, 6, 0, 0):
        extension = "grib2"
        var_name = "HGT_P0_L100_GLL0"
        latlon_suffix = 0

    # generate target file name
    time_string = data_time.strftime("%Y%m%d_%H_%M")
    filename = "./geop/fnl_{0}.{1}".format(time_string, extension)
    # check file existence: if no such file, go to next
    if not os.path.isfile(filename):
        # print "Cannot open file {0}".format(filename)
        num_ng += 1
        data_time += delta_fixee
        continue
        
    # open files and calculate target predictors
    
    # print "Opening file {0}".format(filename)
    geop = Nio.open_file(filename, mode='r')

    wnpsh = geop.variables[var_name]["lat_{0}|10:60 lon_{0}|100:180".format(latlon_suffix)]
    area_index = np.count_nonzero(wnpsh > 5870.0)
    # print "Area index:", area_index
    intensity_index = 0 if area_index == 0 else np.average(wnpsh, weights=(wnpsh > 5870.0))    
    # print "Intensity index:", intensity_index

    everything = geop.variables[var_name].get_value()
    extension_index = 0 if area_index == 0 else min(np.argwhere(np.any(everything > 5870.0, axis=0))) + 53
    extension_index = int(extension_index)
    #print "Westward extension index:", int(extension_index)

    westerly = geop.variables[var_name]["lat_{0}|35,55 lon_{0}|100:180".format(latlon_suffix)]
    westerly_index = np.average(westerly[0] - westerly[1])
    # print "Westerly index:", westerly_index

    geop.close()
    
    # bookkeeping
    wnpsh_area_indices[time_string] = area_index
    wnpsh_intensity_indices[time_string] = intensity_index
    wnpsh_extension_indices[time_string] = extension_index
    westerly_indices[time_string] = westerly_index
    
    num_ok += 1
    data_time += delta_fixee
    
    if num_expected % 1000 == 0:
        print "Completed {0} data files out of {1} expected. {2} files are missing. Current year {3}".format(num_ok, num_expected, num_ng, data_time.year)

Completed 1335 data files out of 13000 expected. 11665 files are missing. Current year 2008
Completed 2335 data files out of 14000 expected. 11665 files are missing. Current year 2009
Completed 3335 data files out of 15000 expected. 11665 files are missing. Current year 2009
Completed 4335 data files out of 16000 expected. 11665 files are missing. Current year 2010
Completed 5335 data files out of 17000 expected. 11665 files are missing. Current year 2011
Completed 6335 data files out of 18000 expected. 11665 files are missing. Current year 2011
Completed 7335 data files out of 19000 expected. 11665 files are missing. Current year 2012
Completed 8335 data files out of 20000 expected. 11665 files are missing. Current year 2013
Completed 9335 data files out of 21000 expected. 11665 files are missing. Current year 2013
Completed 10335 data files out of 22000 expected. 11665 files are missing. Current year 2014
Completed 11335 data files out of 23000 expected. 11665 files are missing. Curr

In [92]:
print num_ng, num_ok, num_expected

print len(wnpsh_area_indices)
print len(wnpsh_intensity_indices)
print len(wnpsh_extension_indices)
print len(westerly_indices)

11665 18167 29832
18167
18167
18167
18167


In [93]:
save_dict_to_csv(wnpsh_area_indices, "wnpsh_area_indices.csv")
save_dict_to_csv(wnpsh_intensity_indices, "wnpsh_intensity_indices.csv")
save_dict_to_csv(wnpsh_extension_indices, "wnpsh_extension_indices.csv")
save_dict_to_csv(westerly_indices, "westerly_indices.csv")

### Winds, u- and v- components, Part 1

This part focuses on the items possible to compute in advance, namely:

- Hong Kong surface winds (1000mb level as proxy):
  - Given HK's coordinates are 22.30N and 114.17E, the nearest 4 grid points should be selected, and then have their values averaged by some appropriate weights, i.e. by interpolation.
  
- East Asia Summer Monsoon (EASM) index:
  - U850 in (5°–15°N, 90°–130°E) minus U850 in (22.5°–32.5°N, 110°–140°E) 
  
  - due to the limited resolution of the data grid, the latter term will be taken for 23N to 33N instead. 
  
Note that this time around there are 11540 GRIB1 files and 17634 GRIB2 files (total 29174) available.

In [97]:
# bookkeeping
hk_u_winds = dict()
hk_v_winds = dict()
easm_indices = dict()

In [98]:
# start from this date
data_time = datetime.datetime(1999, 8, 1, 0, 0, 0)
# advance at 6 hour time steps
delta_fixee = datetime.timedelta(hours=6)

num_ok = 0
num_expected = 0
num_ng = 0

# main loop
# repeat until all data are exhausted (time limit reached)
while data_time < datetime.datetime(2019, 12, 31, 23, 0, 0):
    
    num_expected += 1

    # GRIB1 files are available until 6 Dec 2007 0600 (UTC), 
    # the variables we need in that case are U_GRD_3_ISBL and V_GRD_3_ISBL
    # modify accordingly for GRIB2
    extension = "grib1"
    u_var_name = "U_GRD_3_ISBL"
    v_var_name = "V_GRD_3_ISBL"
    latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
    if data_time > datetime.datetime(2007, 12, 6, 6, 0, 0):
        extension = "grib2"
        u_var_name = "UGRD_P0_L100_GLL0"
        v_var_name = "VGRD_P0_L100_GLL0"
        latlon_suffix = 0

    # generate target file name
    time_string = data_time.strftime("%Y%m%d_%H_%M")
    filename = "./wind/fnl_{0}.{1}".format(time_string, extension)
    # check file existence: if no such file, go to next
    if not os.path.isfile(filename):
        # print "Cannot open file {0}".format(filename)
        num_ng += 1
        data_time += delta_fixee
        continue

    # open files and calculate target predictors

    # print "Opening file {0}".format(filename)
    wind = Nio.open_file(filename, mode='r')

    # hong kong u wind
    hk_u = wind.variables[u_var_name]["lat_{0}|22.30i lon_{0}|114.17i lv_ISBL0|1000".format(latlon_suffix)]
    # print "Hong Kong u-wind:", hk_u

    # hong kong v wind
    hk_v = wind.variables[v_var_name]["lat_{0}|22.30i lon_{0}|114.17i lv_ISBL0|1000".format(latlon_suffix)]
    # print "Hong Kong v-wind:", hk_v

    # easm index
    u850_1 = wind.variables[u_var_name]["lv_ISBL0|850 lon_{0}|90:130".format(latlon_suffix)][(8+5):(8+15),:]
    u850_2 = wind.variables[u_var_name]["lv_ISBL0|850 lon_{0}|110:140".format(latlon_suffix)][(8+23):(8+33),:]
    easm_idx = np.average(u850_1) - np.average(u850_2)
    # print "EASM index:", easm_idx

    wind.close()
    
    # bookkeeping
    hk_u_winds[time_string] = hk_u
    hk_v_winds[time_string] = hk_v
    easm_indices[time_string] = easm_idx
    
    num_ok += 1
    data_time += delta_fixee
    
    if num_expected % 1000 == 0:
        print "Completed {0} data files out of {1} expected. {2} files are missing. Current year {3}".format(num_ok, num_expected, num_ng, data_time.year)

Completed 859 data files out of 1000 expected. 141 files are missing. Current year 2000
Completed 1853 data files out of 2000 expected. 147 files are missing. Current year 2000
Completed 2407 data files out of 3000 expected. 593 files are missing. Current year 2001
Completed 3361 data files out of 4000 expected. 639 files are missing. Current year 2002
Completed 4361 data files out of 5000 expected. 639 files are missing. Current year 2003
Completed 5361 data files out of 6000 expected. 639 files are missing. Current year 2003
Completed 6361 data files out of 7000 expected. 639 files are missing. Current year 2004
Completed 7361 data files out of 8000 expected. 639 files are missing. Current year 2005
Completed 8360 data files out of 9000 expected. 640 files are missing. Current year 2005
Completed 9360 data files out of 10000 expected. 640 files are missing. Current year 2006
Completed 10345 data files out of 11000 expected. 655 files are missing. Current year 2007
Completed 11342 dat

In [99]:
print num_ng, num_ok, num_expected

print len(hk_u_winds)
print len(hk_v_winds)
print len(easm_indices)

658 29174 29832
29174
29174
29174


In [100]:
save_dict_to_csv(hk_u_winds, "hk_u_winds.csv")
save_dict_to_csv(hk_v_winds, "hk_v_winds.csv")
save_dict_to_csv(easm_indices, "easm_indices.csv")

### Read best track dataset

This is to calculate the rest of the variables. The temporal subset 1999-08-01 00:00 to 2019-12-31 23:00 shall be selected. Then, for each TC (identified by SID) and each time in the TC sequence, we calculate the predictors if the corresponding file(s) exist.

In [3]:
best_track = np.load("../best_track.npy", allow_pickle=True)
best_track = pd.DataFrame(best_track)
best_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53791 entries, 0 to 53790
Data columns (total 25 columns):
SID               53791 non-null object
SEASON            53791 non-null int32
NAME              53791 non-null object
ISO_TIME_MONTH    53791 non-null int32
ISO_TIME_DAY      53791 non-null int32
ISO_TIME_HOUR     53791 non-null int32
ISO_TIME_MIN      53791 non-null int32
USA_LAT           53791 non-null float32
USA_LON           53791 non-null float32
USA_WIND          53791 non-null int32
USA_PRES          53791 non-null int32
USA_R34_NE        53791 non-null int32
USA_R34_SE        53791 non-null int32
USA_R34_SW        53791 non-null int32
USA_R34_NW        53791 non-null int32
USA_R50_NE        53791 non-null int32
USA_R50_SE        53791 non-null int32
USA_R50_SW        53791 non-null int32
USA_R50_NW        53791 non-null int32
USA_R64_NE        53791 non-null int32
USA_R64_SE        53791 non-null int32
USA_R64_SW        53791 non-null int32
USA_R64_NW        53791 non

In [4]:
best_track = best_track.query("(SEASON > 1999) or ((SEASON == 1999) and (ISO_TIME_MONTH > 7))")
best_track.describe()

,SEASON,ISO_TIME_MONTH,ISO_TIME_DAY,ISO_TIME_HOUR,ISO_TIME_MIN,USA_LAT,USA_LON,USA_WIND,USA_PRES,USA_R34_NE,...,USA_R50_NE,USA_R50_SE,USA_R50_SW,USA_R50_NW,USA_R64_NE,USA_R64_SE,USA_R64_SW,USA_R64_NW,STORM_SPEED,STORM_DIR
count,15033.000000,15033.000000,15033.000000,15033.000000,15033.0,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,...,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000
mean,2009.038914,8.160247,15.702122,8.980975,0.0,18.981800,131.691406,54.421938,-7429.102441,-43797.865562,...,-67676.852325,-67698.088539,-67872.633806,-67797.964146,-77832.682898,-77899.752944,-77953.541209,-77912.947316,10.168496,234.193441
std,6.187875,2.341304,8.666252,6.716750,0.0,7.349058,19.086857,32.503149,27902.697658,49680.806222,...,46793.468148,46784.069208,46715.952033,46746.058339,41548.057650,41502.381977,41465.478383,41493.640483,5.443846,113.888762
min,1999.000000,1.000000,1.000000,0.000000,0.0,1.300000,-180.000000,10.000000,-99999.000000,-99999.000000,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,0.000000,0.000000
25%,2003.000000,7.000000,8.000000,2.000000,0.0,13.600000,121.300003,30.000000,956.000000,-99999.000000,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,6.000000,191.000000
50%,2009.000000,8.000000,16.000000,6.000000,0.0,18.400000,130.899994,45.000000,987.000000,60.000000,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,9.000000,284.000000
75%,2015.000000,10.000000,23.000000,12.000000,0.0,23.600000,142.000000,75.000000,1000.000000,120.000000,...,36.000000,35.000000,30.000000,35.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,13.000000,310.000000
max,2019.000000,12.000000,31.000000,23.000000,0.0,45.099998,179.800003,170.000000,1012.000000,330.000000,...,200.000000,215.000000,205.000000,196.000000,135.000000,120.000000,110.000000,107.000000,52.000000,360.000000


In [5]:
best_track.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15033 entries, 38758 to 53790
Data columns (total 25 columns):
SID               15033 non-null object
SEASON            15033 non-null int32
NAME              15033 non-null object
ISO_TIME_MONTH    15033 non-null int32
ISO_TIME_DAY      15033 non-null int32
ISO_TIME_HOUR     15033 non-null int32
ISO_TIME_MIN      15033 non-null int32
USA_LAT           15033 non-null float32
USA_LON           15033 non-null float32
USA_WIND          15033 non-null int32
USA_PRES          15033 non-null int32
USA_R34_NE        15033 non-null int32
USA_R34_SE        15033 non-null int32
USA_R34_SW        15033 non-null int32
USA_R34_NW        15033 non-null int32
USA_R50_NE        15033 non-null int32
USA_R50_SE        15033 non-null int32
USA_R50_SW        15033 non-null int32
USA_R50_NW        15033 non-null int32
USA_R64_NE        15033 non-null int32
USA_R64_SE        15033 non-null int32
USA_R64_SW        15033 non-null int32
USA_R64_NW        15033

### Winds, u- and v-components, Part 2

The following should be calculated for each record in the subset:

- Vertical wind shear
  - Upper level, 850mb vs 200mb, 7 degree average
  
  - Mid level, 500mb vs 850mb, 7 degree average
  
- U200, 7 degrees

- U500 and V500, 7 degrees

No more than 15033 values for each variable should be available.

In [140]:
u200_values = dict()
u500_values = dict()
v500_values = dict()
ulvws_values = dict() # Upper-Lower levels Vertical Wind Shear magnitudes
mlvws_values = dict() # Middle-Lower levels Vertical Wind Shear magnitudes

In [141]:
num_ok = 0
num_expected = len(best_track)
num_ng = 0
num_gribs_browsed = 0

for i in range(len(best_track)):
    # select record
    record = best_track.iloc[i]
    timestamp = datetime.datetime(record["SEASON"], record["ISO_TIME_MONTH"], record["ISO_TIME_DAY"], record["ISO_TIME_HOUR"], 0, 0)
    time_string = timestamp.strftime("%Y%m%d_%H_%M")
    key_name = "{0}_{1}".format(record["SID"], time_string)
    # print "Processing TC:", key_name
    
    # many best track records are taken at hours not divisible by 6 (e.g. at 1500)
    # interpolation is needed to maximize the utilization of our weather data
    # loop start: the GRIB files {loop_start} hours back should be considered
    loop_start = 0 if record["ISO_TIME_HOUR"] % 6 == 0 else (record["ISO_TIME_HOUR"] - record["ISO_TIME_HOUR"] % 6)
    # loop end: the subsequent file (which is {loop_end - 6} hours later) to consider, 6 if hours divisible by six (no work needed)
    loop_end = 6 if record["ISO_TIME_HOUR"] % 6 == 0 else loop_start + 12
    # interpolation weight of the earlier record
    weight = 1 - (record["ISO_TIME_HOUR"] - (record["ISO_TIME_HOUR"] // 6)*6)/6.0
    
    # update time reference
    number_of_hours_to_go_back = record["ISO_TIME_HOUR"] % 6
    timestamp -= datetime.timedelta(hours=number_of_hours_to_go_back)
    
    u200_list = list()
    u500_list = list()
    v500_list = list()
    ulvws_list = list()
    mlvws_list = list()
    ng_flag = False
    
    for k in range(loop_start, loop_end, 6):
        time_string = timestamp.strftime("%Y%m%d_%H_%M")   

        # prepare before opening file
        extension = "grib1"
        u_var_name = "U_GRD_3_ISBL"
        v_var_name = "V_GRD_3_ISBL"
        latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
        if timestamp > datetime.datetime(2007, 12, 6, 6, 0, 0):
            extension = "grib2"
            u_var_name = "UGRD_P0_L100_GLL0"
            v_var_name = "VGRD_P0_L100_GLL0"
            latlon_suffix = 0

        # test file exists
        filename = "./wind/fnl_{0}.{1}".format(time_string, extension)
        num_gribs_browsed += 1
        if not os.path.isfile(filename):
            # print "Cannot open file {0}".format(filename)
            ng_flag = True
            break

        # open file
        # print "Opening file", filename
        wind = Nio.open_file(filename, mode='r')

        # U200
        center_lat, center_lon = int(round(record["USA_LAT"])), record["USA_LON"]
        # negative longitudes do not go well with the addressing
        if record["USA_LON"] > 0:
            center_lon = int(round(record["USA_LON"]))
        elif int(round(record["USA_LON"])) == -180:
            center_lon = 180
        else:
            center_lon = int((round(record["USA_LON"])) + 360) % 180
        u200 = wind.variables[u_var_name]["lv_ISBL0|200"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        u200_avg = np.average(u200)
        u200_list.append(u200_avg)
        # print "U200, 7 degree average:", u200_avg

        # U500 and V500
        u500 = wind.variables[u_var_name]["lv_ISBL0|500"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        u500_avg = np.average(u500)
        u500_list.append(u500_avg)
        # print "U500, 7 degree average:", u500_avg
        v500 = wind.variables[v_var_name]["lv_ISBL0|500"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        v500_avg = np.average(v500)
        v500_list.append(v500_avg)
        # print "V500, 7 degree average:",  v500_avg

        # vertical wind shear
        # get V200, U850 and V850 first
        v200 = wind.variables[v_var_name]["lv_ISBL0|200"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        u850 = wind.variables[u_var_name]["lv_ISBL0|850"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        v850 = wind.variables[v_var_name]["lv_ISBL0|850"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        # calculate averages
        v200_avg = np.average(v200)
        u850_avg = np.average(u850)
        v850_avg = np.average(v850)
        # upper-lower wind shear
        hi_low_shear_u = u200_avg - u850_avg
        hi_low_shear_v = v200_avg - v850_avg
        hi_low_shear = math.sqrt(hi_low_shear_u ** 2 + hi_low_shear_v ** 2) # magnitude
        ulvws_list.append(hi_low_shear)
        # print "Wind shear (upper/lower levels):", hi_low_shear_u, hi_low_shear_v, hi_low_shear
        # mid-lower wind shear
        mid_low_shear_u = u500_avg - u850_avg
        mid_low_shear_v = v500_avg - v850_avg
        mid_low_shear = math.sqrt(mid_low_shear_u ** 2 + mid_low_shear_v ** 2) # magnitude
        mlvws_list.append(mid_low_shear)
        # print "Wind shear (mid/lower levels):", mid_low_shear_u, mid_low_shear_v, mid_low_shear

        wind.close()    
    
    # do not record values if there are missing GRIB files
    if ng_flag:
        num_ng += 1
        continue
    
    # compute interpolation
    # print record["ISO_TIME_HOUR"], loop_start, loop_end, u200_list
    if len(u200_list) > 1:
        u200_values[key_name] = weight * u200_list[0] + (1-weight) * u200_list[1]
        u500_values[key_name] = weight * u500_list[0] + (1-weight) * u500_list[1]
        v500_values[key_name] = weight * v500_list[0] + (1-weight) * v500_list[1]
        ulvws_values[key_name] = weight * ulvws_list[0] + (1-weight) * ulvws_list[1]
        mlvws_values[key_name] = weight * mlvws_list[0] + (1-weight) * mlvws_list[1]
    else:
        u200_values[key_name] = u200_list[0]
        u500_values[key_name] = u500_list[0]
        v500_values[key_name] = v500_list[0]
        ulvws_values[key_name] = ulvws_list[0]
        mlvws_values[key_name] = mlvws_list[0]
    
    num_ok += 1
    
    if i % 1500 == 0:
        print "Completed {0} TC records out of {1} expected. {2} records are unusable. Current year {3}".format(num_ok, num_expected, num_ng, record["SEASON"])

Completed 1354 data files out of 15033 expected. 147 files are missing. Current year 2001
Completed 2844 data files out of 15033 expected. 157 files are missing. Current year 2003
Completed 4344 data files out of 15033 expected. 157 files are missing. Current year 2004
Completed 5844 data files out of 15033 expected. 157 files are missing. Current year 2006
Completed 7343 data files out of 15033 expected. 158 files are missing. Current year 2009
Completed 8843 data files out of 15033 expected. 158 files are missing. Current year 2011
Completed 10343 data files out of 15033 expected. 158 files are missing. Current year 2013
Completed 11843 data files out of 15033 expected. 158 files are missing. Current year 2015
Completed 13343 data files out of 15033 expected. 158 files are missing. Current year 2018
Completed 14843 data files out of 15033 expected. 158 files are missing. Current year 2019


In [142]:
print num_ng, num_ok, num_expected, num_gribs_browsed

print len(u200_values)
print len(u500_values)
print len(v500_values)
print len(ulvws_values)
print len(mlvws_values)

158 14875 15033 15165
14875
14875
14875
14875
14875


In [143]:
save_dict_to_csv(u200_values, "u200.csv")
save_dict_to_csv(u500_values, "u500.csv")
save_dict_to_csv(v500_values, "v500.csv")
save_dict_to_csv(ulvws_values, "ulvms.csv")
save_dict_to_csv(mlvws_values, "mlvms.csv")

### Relative humidity

Phew, last one was hard, let's do something simpler:

- 750-800mb relative humidity, 7 degrees

- 300-500mb relative humidity, 7 degrees

There are 18167 records (533 GRIB1, 17634 GRIB2) available and 15033 TC records to match, but only 8422 humidity values could be computed.

Something simpler my donkey. Because of the missing files issue, alternative data sources may be needed.

In [68]:
lo_humid_values = dict()
hi_humid_values = dict()

In [69]:
num_ok, num_ng, num_expected, num_gribs_browsed = 0, 0, len(best_track), 0

for i in range(num_expected):
    # select record
    record = best_track.iloc[i]
    timestamp = datetime.datetime(record["SEASON"], record["ISO_TIME_MONTH"], record["ISO_TIME_DAY"], record["ISO_TIME_HOUR"], 0, 0)
    time_string = timestamp.strftime("%Y%m%d_%H_%M")
    key_name = "{0}_{1}".format(record["SID"], time_string)
    # print "Processing TC:", key_name

    center_lat, center_lon = int(round(record["USA_LAT"])), record["USA_LON"]
    # negative longitudes do not go well with the addressing
    if record["USA_LON"] > 0:
        center_lon = int(round(record["USA_LON"]))
    elif int(round(record["USA_LON"])) == -180:
        center_lon = 180
    else:
        center_lon = int((round(record["USA_LON"])) + 360) % 180

    # many best track records are taken at hours not divisible by 6 (e.g. at 1500)
    # interpolation is needed to maximize the utilization of our weather data
    # loop start: the GRIB files {loop_start} hours back should be considered
    loop_start = 0 if record["ISO_TIME_HOUR"] % 6 == 0 else (record["ISO_TIME_HOUR"] - record["ISO_TIME_HOUR"] % 6)
    # loop end: the subsequent file (which is {loop_end - 6} hours later) to consider, 6 if hours divisible by six (no work needed)
    loop_end = 6 if record["ISO_TIME_HOUR"] % 6 == 0 else loop_start + 12
    # interpolation weight of the earlier record
    weight = 1 - (record["ISO_TIME_HOUR"] - (record["ISO_TIME_HOUR"] // 6)*6)/6.0
    
    # update time reference
    number_of_hours_to_go_back = record["ISO_TIME_HOUR"] % 6
    timestamp -= datetime.timedelta(hours=number_of_hours_to_go_back)
    
    hi_humid_list = list()
    lo_humid_list = list()
    ng_flag = False
    
    for k in range(loop_start, loop_end, 6):
        time_string = timestamp.strftime("%Y%m%d_%H_%M")        
        
        # prepare before opening file
        extension = "grib1"
        var_name = "R_H_3_ISBL"
        latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
        if timestamp > datetime.datetime(2007, 12, 6, 6, 0, 0):
            extension = "grib2"
            var_name = "RH_P0_L100_GLL0"
            latlon_suffix = 0

        # test file exists
        filename = "./humid/fnl_{0}.{1}".format(time_string, extension)
        num_gribs_browsed += 1
        if not os.path.isfile(filename):
            # print "Cannot open file {0}".format(filename)
            ng_flag = True
            break

        # open file
        #print "Opening file", filename
        humid = Nio.open_file(filename, mode='r')

        # 300mb through 500mb
        values = list()
        for j in range(300,550,50):
            grid = humid.variables[var_name]["lv_ISBL0|{0}".format(j)][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
            values.append(grid)
        values = np.stack(values)
        hi_humid = np.average(values)
        hi_humid_list.append(hi_humid)

        # 750-800mb
        values = list()
        for j in range(750,800,50):
            grid = humid.variables[var_name]["lv_ISBL0|{0}".format(j)][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
            values.append(grid)
        values = np.stack(values)
        lo_humid_list.append(np.average(values))

        humid.close()
        
        timestamp += datetime.timedelta(hours=6)

    # do not record values if there are missing GRIB files
    if ng_flag:
        num_ng += 1
        continue
    
    # compute interpolation
    # print record["ISO_TIME_HOUR"], loop_start, loop_end, hi_humid_list
    if len(hi_humid_list) > 1:
        hi_humid_values[key_name] = weight * hi_humid_list[0] + (1-weight) * hi_humid_list[1]
        lo_humid_values[key_name] = weight * lo_humid_list[0] + (1-weight) * lo_humid_list[1]
    else:
        hi_humid_values[key_name] = hi_humid_list[0]
        lo_humid_values[key_name] = lo_humid_list[0]

    num_ok += 1

    if i % 1500 == 0:
        print "Completed {0} TC records out of {1} expected. {2} records cannot be used. Current year {3}".format(num_ok, num_expected, num_ng, record["SEASON"])

Completed 1 TC records out of 15033 expected. 0 records cannot be used. Current year 1999
Completed 890 TC records out of 15033 expected. 6611 records cannot be used. Current year 2009
Completed 2390 TC records out of 15033 expected. 6611 records cannot be used. Current year 2011
Completed 3890 TC records out of 15033 expected. 6611 records cannot be used. Current year 2013
Completed 5390 TC records out of 15033 expected. 6611 records cannot be used. Current year 2015
Completed 6890 TC records out of 15033 expected. 6611 records cannot be used. Current year 2018
Completed 8390 TC records out of 15033 expected. 6611 records cannot be used. Current year 2019


In [136]:
print num_ng, num_ok, num_expected, num_gribs_browsed

print len(lo_humid_values)
print len(hi_humid_values)

6611 8422 15033 15165
8422
8422


In [137]:
save_dict_to_csv(lo_humid_values, "lo_humid.csv")
save_dict_to_csv(hi_humid_values, "hi_humid.csv")

### Temperature

- surface, 2 degree average
  - 11540 GRIB1 and 17634 GRIB2 files (total 29174) are available

- 200mb, 7 degree average
  - total 29174 files available, same as surface


In [26]:
surface_temp_values = dict()
temp200_values = dict()

In [27]:
num_ok, num_ng, num_expected, num_gribs_browsed = 0, 0, len(best_track), 0

for i in range(num_expected):
    # select record
    record = best_track.iloc[i]
    timestamp = datetime.datetime(record["SEASON"], record["ISO_TIME_MONTH"], record["ISO_TIME_DAY"], record["ISO_TIME_HOUR"], 0, 0)
    time_string = timestamp.strftime("%Y%m%d_%H_%M")
    key_name = "{0}_{1}".format(record["SID"], time_string)
    # print "Processing TC:", key_name

    center_lat, center_lon = int(round(record["USA_LAT"])), record["USA_LON"]
    # negative longitudes do not go well with the addressing
    if record["USA_LON"] > 0:
        center_lon = int(round(record["USA_LON"]))
    elif int(round(record["USA_LON"])) == -180:
        center_lon = 180
    else:
        center_lon = int((round(record["USA_LON"])) + 360) % 180

    # many best track records are taken at hours not divisible by 6 (e.g. at 1500)
    # interpolation is needed to maximize the utilization of our weather data
    # loop start: the GRIB files {loop_start} hours back should be considered
    loop_start = 0 if record["ISO_TIME_HOUR"] % 6 == 0 else (record["ISO_TIME_HOUR"] - record["ISO_TIME_HOUR"] % 6)
    # loop end: the subsequent file (which is {loop_end - 6} hours later) to consider, 6 if hours divisible by six (no work needed)
    loop_end = 6 if record["ISO_TIME_HOUR"] % 6 == 0 else loop_start + 12
    # interpolation weight of the earlier record
    weight = 1 - (record["ISO_TIME_HOUR"] - (record["ISO_TIME_HOUR"] // 6)*6)/6.0
    
    # update time reference
    number_of_hours_to_go_back = record["ISO_TIME_HOUR"] % 6
    timestamp -= datetime.timedelta(hours=number_of_hours_to_go_back)
    
    surface_temp_list = list()
    temp_200_list = list()
    ng_flag = False
    
    for k in range(loop_start, loop_end, 6):
        time_string = timestamp.strftime("%Y%m%d_%H_%M")        
        
        # prepare before opening file
        extension = "grib1"
        sfc_var_name = "TMP_3_SFC"
        upper_var_name = "TMP_3_ISBL"
        latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
        if timestamp > datetime.datetime(2007, 12, 6, 6, 0, 0):
            extension = "grib2"
            sfc_var_name = "TMP_P0_L1_GLL0"
            upper_var_name = "TMP_P0_L100_GLL0"
            latlon_suffix = 0

        ## calculate surface 2 degree average
        # test file exists
        filename = "./surface/fnl_{0}.{1}".format(time_string, extension)
        num_gribs_browsed += 1
        if not os.path.isfile(filename):
            # print "Cannot open file {0}".format(filename)
            ng_flag = True
            break

        #print "Opening file", filename
        surface = Nio.open_file(filename, mode='r')
        values = surface.variables[sfc_var_name].get_value()[(center_lat-1+8):(center_lat+1+8),(center_lon-1-53):(center_lon+1-53)]
        surface_temp = np.average(values)
        surface_temp_list.append(surface_temp)        
        surface.close()
        
        ## calculate 200 hpa 7 degree average
        # test file exists
        filename = "./temp/fnl_{0}.{1}".format(time_string, extension)
        num_gribs_browsed += 1
        if not os.path.isfile(filename):
            # print "Cannot open file {0}".format(filename)
            ng_flag = True
            break

        #print "Opening file", filename
        temp = Nio.open_file(filename, mode='r')
        values = temp.variables[upper_var_name]["lv_ISBL0|200"][(center_lat-6+8):(center_lat+6+8),(center_lon-6-53):(center_lon+6-53)]
        temp_200 = np.average(values)
        temp_200_list.append(temp_200)        
        temp.close()
        
        timestamp += datetime.timedelta(hours=6)

    # do not record values if there are missing GRIB files
    if ng_flag:
        num_ng += 1
        continue
    
    # compute interpolation
    # print record["ISO_TIME_HOUR"], loop_start, loop_end, surface_temp_list
    if len(surface_temp_list) > 1:
        surface_temp_values[key_name] = weight * surface_temp_list[0] + (1-weight) * surface_temp_list[1]
        temp200_values[key_name] = weight * temp_200_list[0] + (1-weight) * temp_200_list[1]
    else:
        surface_temp_values[key_name] = surface_temp_list[0]
        temp200_values[key_name] = temp_200_list[0]

    num_ok += 1

    if i % 1500 == 0:
        print "Completed {0} TC records out of {1} expected. {2} records cannot be used. Current year {3}".format(num_ok, num_expected, num_ng, record["SEASON"])

Completed 1354 TC records out of 15033 expected. 147 records cannot be used. Current year 2001
Completed 2844 TC records out of 15033 expected. 157 records cannot be used. Current year 2003
Completed 4344 TC records out of 15033 expected. 157 records cannot be used. Current year 2004
Completed 5844 TC records out of 15033 expected. 157 records cannot be used. Current year 2006
Completed 7343 TC records out of 15033 expected. 158 records cannot be used. Current year 2009
Completed 8843 TC records out of 15033 expected. 158 records cannot be used. Current year 2011
Completed 10343 TC records out of 15033 expected. 158 records cannot be used. Current year 2013
Completed 11843 TC records out of 15033 expected. 158 records cannot be used. Current year 2015
Completed 13343 TC records out of 15033 expected. 158 records cannot be used. Current year 2018
Completed 14843 TC records out of 15033 expected. 158 records cannot be used. Current year 2019


In [28]:
print num_ng, num_ok, num_expected, num_gribs_browsed

print len(surface_temp_values)
print len(temp200_values)

158 14875 15033 30172
14875
14875


In [29]:
save_dict_to_csv(surface_temp_values, "temp_surface.csv")
save_dict_to_csv(temp200_values, "temp200.csv")

### Vorticity

850mb relative vorticity (7 degrees average) is needed. The given data is in *absolute* vorticity, where an extra term for Coriolis force is present. This can be manually taken out. However, the resulting values look unreasonably low compared to known relative vorticity values, so absolute vorticity is kept.

18167 data files are available (533 grib1 and 17634 grib2).

Because of the missing files issue, alternative data sources may be needed.

In [57]:
import math

def coriolis_param(lat):
    '''Computes the Coriolis parameter from the given latitude (in degrees)'''
    omega = 7.2921e-05
    return 2*omega*math.sin(math.radians(lat))

vort_values = dict()

num_ok, num_ng, num_expected, num_gribs_browsed = 0, 0, len(best_track), 0

for i in range(num_expected):
    # select record
    record = best_track.iloc[i]
    timestamp = datetime.datetime(record["SEASON"], record["ISO_TIME_MONTH"], record["ISO_TIME_DAY"], record["ISO_TIME_HOUR"], 0, 0)
    time_string = timestamp.strftime("%Y%m%d_%H_%M")
    key_name = "{0}_{1}".format(record["SID"], time_string)
    # print "Processing TC:", key_name

    center_lat, center_lon = int(round(record["USA_LAT"])), record["USA_LON"]
    # negative longitudes do not go well with the addressing
    if record["USA_LON"] > 0:
        center_lon = int(round(record["USA_LON"]))
    elif int(round(record["USA_LON"])) == -180:
        center_lon = 180
    else:
        center_lon = int((round(record["USA_LON"])) + 360) % 180

    # many best track records are taken at hours not divisible by 6 (e.g. at 1500)
    # interpolation is needed to maximize the utilization of our weather data
    # loop start: the GRIB files {loop_start} hours back should be considered
    loop_start = 0 if record["ISO_TIME_HOUR"] % 6 == 0 else (record["ISO_TIME_HOUR"] - record["ISO_TIME_HOUR"] % 6)
    # loop end: the subsequent file (which is {loop_end - 6} hours later) to consider, 6 if hours divisible by six (no work needed)
    loop_end = 6 if record["ISO_TIME_HOUR"] % 6 == 0 else loop_start + 12
    # interpolation weight of the earlier record
    weight = 1 - (record["ISO_TIME_HOUR"] - (record["ISO_TIME_HOUR"] // 6)*6)/6.0
    
    # update time reference
    number_of_hours_to_go_back = record["ISO_TIME_HOUR"] % 6
    timestamp -= datetime.timedelta(hours=number_of_hours_to_go_back)
    
    vort850_list = list()
    ng_flag = False
    
    for k in range(loop_start, loop_end, 6):
        time_string = timestamp.strftime("%Y%m%d_%H_%M")        
        
        # prepare before opening file
        extension = "grib1"
        var_name = "ABS_V_3_ISBL"
        latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
        if timestamp > datetime.datetime(2007, 12, 6, 6, 0, 0):
            extension = "grib2"
            var_name = "ABSV_P0_L100_GLL0"
            latlon_suffix = 0

        ## calculate surface 2 degree average
        # test file exists
        filename = "./vort/fnl_{0}.{1}".format(time_string, extension)
        num_gribs_browsed += 1
        if not os.path.isfile(filename):
            # print "Cannot open file {0}".format(filename)
            ng_flag = True
            break

        #print "Opening file", filename
        vort = Nio.open_file(filename, mode='r')
        values = vort.variables[var_name].get_value()[(center_lat-1+8):(center_lat+1+8),(center_lon-1-53):(center_lon+1-53)]
        vort.close()
        
        '''for x in range(values.shape[0]):
            for y in range(values.shape[1]):
                values[x,y] = values[x,y] - coriolis_param(y + (center_lat-6+8))'''
                
        vort850 = np.average(values)
        vort850_list.append(vort850)
        
        timestamp += datetime.timedelta(hours=6)

    # do not record values if there are missing GRIB files
    if ng_flag:
        num_ng += 1
        continue
    
    # compute interpolation
    # print record["ISO_TIME_HOUR"], loop_start, loop_end, vort850_list
    if len(vort850_list) > 1:
        vort_values[key_name] = weight * vort850_list[0] + (1-weight) * vort850_list[1]
    else:
        vort_values[key_name] = vort850_list[0]

    num_ok += 1

    if i % 1500 == 0:
        print "Completed {0} TC records out of {1} expected. {2} records cannot be used. Current year {3}".format(num_ok, num_expected, num_ng, record["SEASON"])

Completed 1 TC records out of 15033 expected. 0 records cannot be used. Current year 1999
Completed 890 TC records out of 15033 expected. 6611 records cannot be used. Current year 2009
Completed 2390 TC records out of 15033 expected. 6611 records cannot be used. Current year 2011
Completed 3890 TC records out of 15033 expected. 6611 records cannot be used. Current year 2013
Completed 5390 TC records out of 15033 expected. 6611 records cannot be used. Current year 2015
Completed 6890 TC records out of 15033 expected. 6611 records cannot be used. Current year 2018
Completed 8390 TC records out of 15033 expected. 6611 records cannot be used. Current year 2019


In [58]:
print num_ng, num_ok, num_expected, num_gribs_browsed

print len(vort_values)

6611 8422 15033 15165
8422


In [59]:
save_dict_to_csv(vort_values, "vort850.csv")

### Potential Temperature

The original plan is to calculate it for 925mb level, but the data comes in sigma level 0.995, i.e. the pressure level is 0.995 that of the surface pressure. Therefore the 925mb level potential temperature cannot be calculated. Instead, a direct 2-degree average will be taken.

11540 GRIB1 and 17633 GRIB2 files are available, adding up to a total of 29173 files. Interestingly enough, potential temperature is a variable that is rather hard to come by in other atmospheric reanalysis archives.

In [62]:
pott_values = dict()

num_ok, num_ng, num_expected, num_gribs_browsed = 0, 0, len(best_track), 0

for i in range(num_expected):
    # select record
    record = best_track.iloc[i]
    timestamp = datetime.datetime(record["SEASON"], record["ISO_TIME_MONTH"], record["ISO_TIME_DAY"], record["ISO_TIME_HOUR"], 0, 0)
    time_string = timestamp.strftime("%Y%m%d_%H_%M")
    key_name = "{0}_{1}".format(record["SID"], time_string)
    # print "Processing TC:", key_name

    center_lat, center_lon = int(round(record["USA_LAT"])), record["USA_LON"]
    # negative longitudes do not go well with the addressing
    if record["USA_LON"] > 0:
        center_lon = int(round(record["USA_LON"]))
    elif int(round(record["USA_LON"])) == -180:
        center_lon = 180
    else:
        center_lon = int((round(record["USA_LON"])) + 360) % 180

    # many best track records are taken at hours not divisible by 6 (e.g. at 1500)
    # interpolation is needed to maximize the utilization of our weather data
    # loop start: the GRIB files {loop_start} hours back should be considered
    loop_start = 0 if record["ISO_TIME_HOUR"] % 6 == 0 else (record["ISO_TIME_HOUR"] - record["ISO_TIME_HOUR"] % 6)
    # loop end: the subsequent file (which is {loop_end - 6} hours later) to consider, 6 if hours divisible by six (no work needed)
    loop_end = 6 if record["ISO_TIME_HOUR"] % 6 == 0 else loop_start + 12
    # interpolation weight of the earlier record
    weight = 1 - (record["ISO_TIME_HOUR"] - (record["ISO_TIME_HOUR"] // 6)*6)/6.0
    
    # update time reference
    number_of_hours_to_go_back = record["ISO_TIME_HOUR"] % 6
    timestamp -= datetime.timedelta(hours=number_of_hours_to_go_back)
    
    pott_list = list()
    ng_flag = False
    
    for k in range(loop_start, loop_end, 6):
        time_string = timestamp.strftime("%Y%m%d_%H_%M")        
        
        # prepare before opening file
        extension = "grib1"
        var_name = "POT_3_SIGL"
        latlon_suffix = 3 # grib1: lat_3, grib2: lat_0
        if timestamp > datetime.datetime(2007, 12, 6, 6, 0, 0):
            extension = "grib2"
            var_name = "POT_P0_L104_GLL0"
            latlon_suffix = 0

        ## calculate surface 2 degree average
        # test file exists
        filename = "./potential_temp/fnl_{0}.{1}".format(time_string, extension)
        num_gribs_browsed += 1
        if not os.path.isfile(filename):
            # print "Cannot open file {0}".format(filename)
            ng_flag = True
            break

        #print "Opening file", filename
        pott = Nio.open_file(filename, mode='r')
        values = pott.variables[var_name].get_value()[(center_lat-1+8):(center_lat+1+8),(center_lon-1-53):(center_lon+1-53)]
        pott.close()
                
        pott_avg = np.average(values)
        pott_list.append(pott_avg)
        
        timestamp += datetime.timedelta(hours=6)

    # do not record values if there are missing GRIB files
    if ng_flag:
        num_ng += 1
        continue
    
    # compute interpolation
    # print record["ISO_TIME_HOUR"], loop_start, loop_end, pott_list
    if len(pott_list) > 1:
        pott_values[key_name] = weight * pott_list[0] + (1-weight) * pott_list[1]
    else:
        pott_values[key_name] = pott_list[0]

    num_ok += 1

    if i % 1500 == 0:
        print "Completed {0} TC records out of {1} expected. {2} records cannot be used. Current year {3}".format(num_ok, num_expected, num_ng, record["SEASON"])

Completed 1354 TC records out of 15033 expected. 147 records cannot be used. Current year 2001
Completed 2844 TC records out of 15033 expected. 157 records cannot be used. Current year 2003
Completed 4344 TC records out of 15033 expected. 157 records cannot be used. Current year 2004
Completed 5844 TC records out of 15033 expected. 157 records cannot be used. Current year 2006
Completed 7343 TC records out of 15033 expected. 158 records cannot be used. Current year 2009
Completed 8843 TC records out of 15033 expected. 158 records cannot be used. Current year 2011
Completed 10341 TC records out of 15033 expected. 160 records cannot be used. Current year 2013
Completed 11841 TC records out of 15033 expected. 160 records cannot be used. Current year 2015
Completed 13341 TC records out of 15033 expected. 160 records cannot be used. Current year 2018
Completed 14841 TC records out of 15033 expected. 160 records cannot be used. Current year 2019


In [67]:
print num_ng, num_ok, num_expected, num_gribs_browsed

print len(pott_values)

save_dict_to_csv(pott_values, "pott.csv")

160 14873 15033 15165
14873
